In [1]:
import numpy as np
from dataclasses import dataclass
from typing import List
from scipy.special import comb
import itertools

In [112]:
@dataclass
class Monomial:
    name: str
    deg: int
    
@dataclass
class Polynomial:
    name: str
    deg: int
    monomials: List[Monomial]

    def __init__ (self, monomials):
        self.monomials = monomials

    def update(self):
        sorted(self.monomials, key = lambda monom: monom.deg, reverse = True)
        self.deg = max(self.monomials[0].deg, self.monomials[-1].deg)
        self.name = ""
        for i, monom in enumerate(self.monomials):
            self.name += monom.name if i == 0 else " + " + monom.name

@dataclass
class Encode_line:
    data: np.ndarray
    polynomial: Polynomial

    def update(self):
        self.polynomial.update()
        return self
    
    def __str__(self):
        return "{} #|# {}".format(str(self.data.astype("int"))[1:-1], self.polynomial.name)
    
    def __repr__(self):
        return self.__str__()
    
    def get_position(self, point):
        return sum(2**(len(point) - i - 1) for i, bit in enumerate(np.array(point).astype(np.bool)) if bit)

    def get_value(self, point):
        return self.data[self.get_position(point)]

@dataclass
class Noise_line (Encode_line):
    def __init__ (self: object, encode_line: Encode_line, noise_num: int):
        self.data = noise(encode_line.data, noise_num)
        self.polynomial = encode_line.polynomial


def count_k(r: int, m: int):
    return sum(comb(m, i, exact = True) for i in range(r + 1))

class G_matrix():
    def __init__ (self:object, shape: tuple):
        '''
        shape = (k, m) 
        n == 2**m
        '''

        self.k = shape[0]
        self.m = shape[1]
        self.n = 2**shape[1]

        self.data = np.zeros((self.k, self.n),  dtype=np.bool)
        self.monomials = np.ndarray(shape[0], dtype=Monomial)
    
    def __str__ (self):
        result = ""
        for i in range(len(self.data)):
            result += "{} #|# {}\n".format(str(self.data[i].astype("int"))[1:-1], self.monomials[i].name)
        return result
    def __repr__ (self):
        return self.__str__()
    
    def __recur_init(self):
        recur_m = 0

        self.monomials[0] = Monomial('', 0)
        self.data[0][0] = True

        def copy_G_m(dst: tuple):
            for i in range(2**recur_m):
                for j in range(2**recur_m):
                    self.data[dst[0] + i][dst[1] + j] = self.data[i][j]
        def copy_and_update_info():
            for i in range(2**recur_m):
                self.monomials[2**recur_m + i] = Monomial(self.monomials[i].name + "x{}".format(recur_m + 1), self.monomials[i].deg + 1)

        while recur_m < self.m:
            copy_G_m((0, 2**recur_m))
            copy_G_m((2**recur_m, 2**recur_m))
            copy_and_update_info()
            recur_m += 1
        
        self.monomials[0].name = '1'
        return self

    @classmethod
    def full(cls:type, m: int):
        return cls(shape = (2**m, m)).__recur_init()

    @classmethod
    def truncate(cls:type, r: int, m: int):
        full_G = cls.full(m)

        if r == m:
            return full_G

        k = count_k(r, m)
        truncated_G = cls(shape = (k, m))
        j = 0
        for i in range(2**m):
            if full_G.monomials[i].deg <= r:
                truncated_G.data[j] = full_G.data[i]
                truncated_G.monomials[j] = full_G.monomials[i]
                j += 1
            if j == k:
                break
        return truncated_G


**G_matrix_debug**

In [113]:
m = 3
r = 1

full_example = G_matrix.full(m = m)
full_example

1 1 1 1 1 1 1 1 #|# 1
0 1 0 1 0 1 0 1 #|# x1
0 0 1 1 0 0 1 1 #|# x2
0 0 0 1 0 0 0 1 #|# x1x2
0 0 0 0 1 1 1 1 #|# x3
0 0 0 0 0 1 0 1 #|# x1x3
0 0 0 0 0 0 1 1 #|# x2x3
0 0 0 0 0 0 0 1 #|# x1x2x3

In [114]:
m = 3
r = 1

trancate_example = G_matrix.truncate(r = r, m = m)
trancate_example

1 1 1 1 1 1 1 1 #|# 1
0 1 0 1 0 1 0 1 #|# x1
0 0 1 1 0 0 1 1 #|# x2
0 0 0 0 1 1 1 1 #|# x3

In [115]:
def noise(array:np.ndarray, noise_num:int):
    array_l = len(array)
    if array_l < noise_num:
        print("Error! Array less than noise_num")
        return
    choice = np.random.choice(len(array), noise_num, replace = False)
    result_array = array.copy()
    for i in choice:
        result_array[i] = not result_array[i]
    return result_array

# test = np.random.choice(a=[False, True], size=(10, ))
# print(test, noise(test, 2))

class RM_encoder():
    def __init__ (self: object, r: int, m: int):
        self.generator = G_matrix.truncate(r, m)
        self.r = r
        self.m = m
        self.t = 2**(self.m - self.r - 1) - 1
    def encode (self, input):
        input_as_bool = np.array(input).astype(np.bool)

        data_numeric = np.dot(input_as_bool.astype(np.int), self.generator.data.astype(np.int))
        data = np.zeros(self.generator.n, dtype = np.bool)
        for i, val in enumerate(data_numeric):
            if (data_numeric[i] % 2) == 1:
                data[i] = True

        monomials = []
        for i, bit in enumerate(np.array(input).astype(np.bool)):
            if bit:
                monomials.append(self.generator.monomials[i])
        return Encode_line(data, Polynomial(monomials)).update()
    
    def encode_with_noise (self, input, noise_num: int):
        encode_line = self.encode(input)
        if noise_num > self.t:
            print("Warning! Max noise is {}".format(self.t))
        
        return Noise_line(encode_line, noise_num)

**Encoder and Noise debug**

In [116]:
m = 3
r = 1

rm_encoder = RM_encoder(r = r, m = m)

pure_encode = rm_encoder.encode([1, 1, 1, 1])
print(pure_encode)
print(rm_encoder.encode_with_noise([1, 1, 1, 1], 1))
print(rm_encoder.encode_with_noise([1, 1, 1, 1], 2))

1 0 0 1 0 1 1 0 #|# 1 + x1 + x2 + x3
1 0 0 1 0 1 0 0 #|# 1 + x1 + x2 + x3
Warning! Max noise is 1
1 0 1 1 0 1 1 1 #|# 1 + x1 + x2 + x3


In [117]:
pure_encode = rm_encoder.encode([1, 1, 1, 1])

print(pure_encode.get_value([0, 0, 0]))
print(pure_encode.get_value([0, 0, 1]))
print(pure_encode.get_value([0, 1, 0]))
print(pure_encode.get_value([0, 1, 1]))
print(pure_encode.get_value([1, 0, 0]))
print(pure_encode.get_value([1, 0, 1]))
print(pure_encode.get_value([1, 1, 0]))
print(pure_encode.get_value([1, 1, 1]))

True
False
False
True
False
True
True
False


**Noise Encoder debug**

In [133]:
def point_from_position (result: np.ndarray, position: int):
    for i in range(len(result)):
        val = 2**(len(result) - i - 1)
        if position >= val:
            result[i] = True
            position -= val
        else:
            result[i] = False
    return result

class RM_decoder():
    def __init__ (self: object, r: int, m: int):
        self.r = r
        self.m = m
    def decode (self: object, encode: Encode_line):
        recur_r = self.r
        result = []
        while recur_r >= 0:
            self.__recur_decode(encode, recur_r, result)
            recur_r -= 1
        return result
    def __recur_decode (self: object, encode: Encode_line, recur_r: int, result :list):
        
        b_points = np.zeros(self.m - recur_r, dtype = np.bool)
        x_points = np.zeros(recur_r, dtype = np.bool)

        full_points = np.zeros(self.m, dtype = np.bool)

        for monom_arg in itertools.combinations(np.arange(self.m), recur_r):
            monom_arg_mask = np.zeros(self.m, dtype = np.bool)
            for arg in monom_arg:
                monom_arg_mask[arg] = True

            fix_b = 0
            fix_x = 0

            fix_arg_values = np.zeros(2, dtype = np.int)
            for fix_b in range(2**(self.m - recur_r)):
                b_points = point_from_position (b_points, fix_b)
                fix_b_value = 0
                for fix_x in range(2**recur_r):
                    x_points = point_from_position (x_points, fix_x)
                    b_i = 0
                    x_i = 0
                    for mask_i, mask_value in enumerate(monom_arg_mask):
                        if mask_value == False:
                            full_points[mask_i] = b_points[b_i]
                            b_i += 1 
                        else:
                            full_points[mask_i] = x_points[x_i]
                            x_i += 1
                    #print(full_points, x_points, b_points, monom_arg_mask)
                    # print(encode.get_value(full_points),encode.get_position(full_points),  full_points)
                    fix_b_value += int(encode.get_value(full_points))
                fix_arg_values [fix_b_value % 2] += 1
            if fix_arg_values[1] > fix_arg_values[0]:
                result.append((monom_arg, 1))
                for i in range(2**self.m):
                    full_points = point_from_position (full_points, i)
                    change_flag = True
                    for mask_i, mask_value in enumerate(monom_arg_mask):
                        if mask_value == True and full_points[mask_i] != True:
                            change_flag = False
                            break
                    if change_flag:
                        encode.data[i] = not encode.data[i]
            else:
                result.append((monom_arg, 0))
            # print(encode.data, fix_arg_values)

In [134]:
r = 1
m = 4

rm_encoder = RM_encoder(r = r, m = m)
pure_encode = rm_encoder.encode([1, 1, 1, 1, 1])

rm_decoder = RM_decoder(r = r, m = m)

In [135]:
rm_decoder.decode(rm_encoder.encode_with_noise([1, 1, 1, 1, 1], 3))

[((0,), 1), ((1,), 1), ((2,), 1), ((3,), 1), ((), 1)]